# Model creation using XGBOOST


In [2]:
# open parquet file
import pandas as pd

path = "/Users/macbook/Development/sun_eu/data/03_processed/brandenburger_tor_hourly_cleaned_features.parquet"
df = pd.read_parquet(path)


In [3]:
print(df.info())
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35061 entries, 0 to 35060
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   timestamp                   35061 non-null  datetime64[ns]
 1   local_timestamp             35061 non-null  datetime64[ns]
 2   year                        35061 non-null  int32         
 3   month                       35061 non-null  int32         
 4   day                         35061 non-null  int32         
 5   dayofweek                   35061 non-null  int32         
 6   hour                        35061 non-null  int32         
 7   power_output_lead_1h        35061 non-null  float64       
 8   power_output                35061 non-null  float64       
 9   power_output_lag_1hr        35061 non-null  float64       
 10  power_output_lag_2hr        35061 non-null  float64       
 11  direct_irradiance           35061 non-null  float64   

,timestamp,local_timestamp,year,month,day,dayofweek,hour,power_output_lead_1h,power_output,power_output_lag_1hr,...,diffuse_irradiance_lag_1hr,diffuse_irradiance_lag_2hr,sun_height,sun_height_lag_1hr,sun_height_lag_2hr,air_temperature,air_temperature_lag_1hr,wind_speed_at_10m,wind_speed_at_10m_lag_1hr,wind_speed_at_10m_lag_2hr
count,35061,35061,35061.000000,35061.000000,35061.000000,35061.000000,35061.000000,35061.000000,35061.000000,35061.000000,...,35061.00000,35061.00000,35061.000000,35061.000000,35061.000000,35061.000000,35061.000000,35061.000000,35061.000000,35061.000000
mean,2021-12-31 12:10:59.999999488,2021-12-31 13:46:21.742106112,2021.499016,6.523088,15.730042,4.002139,11.500299,452.418256,452.418256,452.418256,...,64.16222,64.16222,12.511930,12.511930,12.511930,10.926813,10.926697,2.996454,2.996440,2.996414
min,2020-01-01 02:11:00,2020-01-01 03:11:00,2020.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.000000,-15.770000,-15.770000,0.000000,0.000000,0.000000
25%,2020-12-31 07:11:00,2020-12-31 08:11:00,2020.000000,4.000000,8.000000,2.000000,6.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.000000,4.660000,4.660000,2.000000,2.000000,2.000000
50%,2021-12-31 12:11:00,2021-12-31 13:11:00,2021.000000,7.000000,16.000000,4.000000,12.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.000000,10.340000,10.340000,2.760000,2.760000,2.760000
75%,2022-12-31 17:11:00,2022-12-31 18:11:00,2022.000000,10.000000,23.000000,6.000000,17.000000,620.600000,620.600000,620.600000,...,109.00000,109.00000,22.030000,22.030000,22.030000,17.000000,17.000000,3.720000,3.720000,3.720000
max,2023-12-31 22:11:00,2023-12-31 23:11:00,2023.000000,12.000000,31.000000,7.000000,23.000000,3308.540000,3308.540000,3308.540000,...,434.00000,434.00000,60.920000,60.920000,60.920000,37.480000,37.480000,11.030000,11.030000,11.030000
std,NaN,NaN,1.118317,3.448524,8.799513,2.000562,6.921809,750.353301,750.353301,750.353301,...,94.58708,94.58708,17.086078,17.086078,17.086078,8.057049,8.057156,1.412412,1.412415,1.412419


## Prepare test, train & validation data

In [4]:
features = list(df.drop(["power_output_lead_1h","timestamp","local_timestamp"], axis=1).columns)
target = "power_output_lead_1h"

X = df[features]
y = df[target]

print(X.shape, type(X))
print(y.shape)

(35061, 22) <class 'pandas.core.frame.DataFrame'>
(35061,)


## Training model using XGBoost
- Use TimeSeriesSplit for cross-validation
- tune hyperparameters using RandomizedSearchCV
- run with a pipeline
- train using the best hyperparameters

In [5]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, mean_squared_error 
import time 

print(f"Feature shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Define lists for hyperparameters
params = {
    #Pipeline step name ('xgb') + '__' + parameter name
    'xgb__n_estimators': [int(x) for x in np.linspace(start=200, stop=1200, num=6)], # Sample from a list
    'xgb__learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'xgb__max_depth': [3, 4, 5, 6, 7, 8], 
    'xgb__subsample': [0.6, 0.7, 0.8, 0.9, 1.0], 
    'xgb__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0], 
    'xgb__gamma': [0, 0.1, 0.2, 0.3], # Minimum loss reduction
    'xgb__reg_alpha': [0, 0.01, 0.1, 1], # L1 regularization
    'xgb__reg_lambda': [0.1, 1, 1.5, 2] # L2 regularization
}

# Setup TimeSeries Split
n_splits = 5 
tscv = TimeSeriesSplit(n_splits=n_splits)

# Create Pipeline 
# 'scaler' and 'xgb' are arbitrary names for the steps
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', xgb.XGBRegressor(objective='reg:squarederror',
                             random_state=42,
                             n_jobs=-1)) # Set fixed parameters here
])

#  Scoring Metric
scoring_metric = 'neg_root_mean_squared_error'  # We want to minimize RMSE, so use neg_root_mean_squared_error (higher is better)

# Instantiate RandomizedSearchCV 
n_iter = 50 # Number of parameter settings that are sampled. Increase for better search, decrease for speed.

random_search = RandomizedSearchCV(
    estimator=pipeline,            # Use the pipeline
    param_distributions=params,    # Parameter space to sample from
    n_iter=n_iter,                 # Number of iterations/combinations to try
    scoring=scoring_metric,       # Metric to optimize (negative RMSE)
    cv=tscv,                       # Use TimeSeriesSplit for cross-validation!
    n_jobs=1,                     # Use all available CPU cores
    verbose=2,                     # Show progress (0=silent, 1=basic, 2=detailed)
    random_state=42                # For reproducible sampling
)
print(f"RandomizedSearchCV setup with {n_iter} iterations.")

# Run the Search 
print("Starting hyperparameter search...")
start_time = time.time()
random_search.fit(X, y) # Fit on the full X and y data
end_time = time.time()
print(f"Search finished in {(end_time - start_time):.2f} seconds.")

# Analyze Results
print("\n--- Hyperparameter Tuning Results ---")
print(f"Best Score (Negative RMSE): {random_search.best_score_:.4f}")
# Convert back to positive RMSE:
print(f"Best Positive RMSE: {-random_search.best_score_:.4f}")

print("\nBest Parameters Found:")
# Best parameters are prefixed with the pipeline step name ('xgb__')
best_params_raw = random_search.best_params_
print(best_params_raw)


Feature shape: (35061, 22)
Target shape: (35061,)
RandomizedSearchCV setup with 50 iterations.
Starting hyperparameter search...
Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END xgb__colsample_bytree=0.8, xgb__gamma=0, xgb__learning_rate=0.1, xgb__max_depth=5, xgb__n_estimators=200, xgb__reg_alpha=0.01, xgb__reg_lambda=2, xgb__subsample=0.9; total time=   0.3s
[CV] END xgb__colsample_bytree=0.8, xgb__gamma=0, xgb__learning_rate=0.1, xgb__max_depth=5, xgb__n_estimators=200, xgb__reg_alpha=0.01, xgb__reg_lambda=2, xgb__subsample=0.9; total time=   0.3s
[CV] END xgb__colsample_bytree=0.8, xgb__gamma=0, xgb__learning_rate=0.1, xgb__max_depth=5, xgb__n_estimators=200, xgb__reg_alpha=0.01, xgb__reg_lambda=2, xgb__subsample=0.9; total time=   0.4s
[CV] END xgb__colsample_bytree=0.8, xgb__gamma=0, xgb__learning_rate=0.1, xgb__max_depth=5, xgb__n_estimators=200, xgb__reg_alpha=0.01, xgb__reg_lambda=2, xgb__subsample=0.9; total time=   0.4s
[CV] END xgb__colsample_bytree=0.

In [19]:
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error
import pandas as pd
import numpy as np # Make sure numpy is imported


# best parameters found from RandomizedSearchCV
best_params_xgb = {
    'n_estimators': 400,
    'learning_rate': 0.01,
    'max_depth': 5,
    'subsample': 0.9,
    'colsample_bytree': 0.7,
    'gamma': 0.2,
    'reg_alpha': 0.1,
    'reg_lambda': 2,
    'objective': 'reg:squarederror',
    'random_state': 42,             
    'n_jobs': -1                  
}

# split data into train and test sets

split_fraction = 0.8 
split_index = int(len(X) * split_fraction)

X_train = X.iloc[:split_index]
y_train = y.iloc[:split_index]

X_test = X.iloc[split_index:]
y_test = y.iloc[split_index:]


# Create final pipeline
final_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', xgb.XGBRegressor(
        **best_params_xgb
    ))
])

final_pipeline.fit(X_train, y_train)
print("Final model training complete.")

# Evaluate the Final Pipeline on the  Test Set
y_pred_final = final_pipeline.predict(X_test)

# Store results from each fold
rmse_scores = []
mae_scores = []
medae_scores = []
r2_scores = []


# Final performance metrics
final_rmse = np.sqrt(mean_squared_error(y_test, y_pred_final))
final_mae = mean_absolute_error(y_test, y_pred_final)
final_medae = median_absolute_error(y_test, y_pred_final)
final_r2 = r2_score(y_test, y_pred_final)

print(f"--- Final Results: ---")
print(f"    RMSE: {final_rmse:.4f}")
print(f"    MAE:  {final_mae:.4f}")
print(f"    MedianAE: {final_medae:.4f}")
print(f"    R²:   {final_r2:.4f}")

# Append scores
rmse_scores.append(final_rmse)
mae_scores.append(final_mae)
medae_scores.append(final_medae)
r2_scores.append(final_r2)

print(f"\nCompare Hold-out RMSE ({final_rmse:.4f}) to CV Best Avg RMSE ({-random_search.best_score_:.4f})")


# Average Results 
print("\n--- Average Final results---")
print(f"  Average RMSE: {np.mean(rmse_scores):.4f} (+/- {np.std(rmse_scores):.4f})")
print(f"  Average MAE:  {np.mean(mae_scores):.4f} (+/- {np.std(mae_scores):.4f})")
print(f"  Average MedianAE: {np.mean(medae_scores):.4f} (+/- {np.std(medae_scores):.4f})")
print(f"  Average R²:   {np.mean(r2_scores):.4f} (+/- {np.std(r2_scores):.4f})")

Final model training complete.
--- Final Results: ---
    RMSE: 238.9622
    MAE:  119.0843
    MedianAE: 22.2576
    R²:   0.9126

Compare Hold-out RMSE (238.9622) to CV Best Avg RMSE (211.8630)

--- Average Final results---
  Average RMSE: 238.9622 (+/- 0.0000)
  Average MAE:  119.0843 (+/- 0.0000)
  Average MedianAE: 22.2576 (+/- 0.0000)
  Average R²:   0.9126 (+/- 0.0000)


## Features importance

In [20]:
# Get importances from the model

xgb_model = final_pipeline.named_steps['xgb']
if isinstance(X_train, pd.DataFrame):
    feature_names = X_train.columns.tolist()

importances = xgb_model.feature_importances_

fi = pd.DataFrame(data=importances,
             index=feature_names,
             columns=['importance']
            )           
fi_sorted = round(fi.head(10),3)
fi_sorted

,importance
year,0.001
month,0.007
day,0.002
dayofweek,0.002
hour,0.035
power_output,0.539
power_output_lag_1hr,0.033
power_output_lag_2hr,0.010
direct_irradiance,0.060
direct_irradiance_lag_1hr,0.021


In [21]:
import plotly.express as px

fi_hist = px.bar(fi_sorted.sort_values('importance',ascending=True),
                orientation='h',
                title=f'Feature Importances', 
                template='plotly_dark'  
                )

fi_hist.show()

### Errors analysis

In [26]:
# histogramme of error prediction
import plotly.express as px

y_pred = final_pipeline.predict(X_test)

err_hist = np.abs(y_test - y_pred)

err_plot = px.histogram(
                    err_hist,
                    template='plotly_dark',
                    nbins=50
            )

err_plot.show()

The model performs pretty well most of the time (low MedianAE), 
but occasionally makes substantial errors that significantly impact the average error scores.


## Prediction comparison

In [10]:
# create index from original df
#df_index = df.reset_index()
df_index = df.set_index('timestamp')
df_index.head(5)


,local_timestamp,year,month,day,dayofweek,hour,power_output_lead_1h,power_output,power_output_lag_1hr,power_output_lag_2hr,...,diffuse_irradiance_lag_1hr,diffuse_irradiance_lag_2hr,sun_height,sun_height_lag_1hr,sun_height_lag_2hr,air_temperature,air_temperature_lag_1hr,wind_speed_at_10m,wind_speed_at_10m_lag_1hr,wind_speed_at_10m_lag_2hr
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-01-01 02:11:00,2020-01-01 03:11:00,2020,1,1,4,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.04,1.48,2.34,2.41,2.34
2020-01-01 03:11:00,2020-01-01 04:11:00,2020,1,1,4,3,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.64,1.04,2.21,2.34,2.41
2020-01-01 04:11:00,2020-01-01 05:11:00,2020,1,1,4,4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.21,0.64,2.14,2.21,2.34
2020-01-01 05:11:00,2020-01-01 06:11:00,2020,1,1,4,5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.06,0.21,2.21,2.14,2.21
2020-01-01 06:11:00,2020-01-01 07:11:00,2020,1,1,4,6,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.27,-0.06,2.21,2.21,2.14


In [11]:
# Create Series with matching index
predictions_series = pd.Series(y_pred, index=df_index.index, name='Predicted')

y_test.shape()
plot_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': predictions_series
})

plot_df = plot_df.reset_index()

timestamp_col_name = plot_df.columns[0]

plot_df_long = pd.melt(
    plot_df,
    id_vars=[timestamp_col_name], # Column(s) to keep fixed (the timestamp)
    var_name='Source',      # Name for the new column indicating 'Actual' or 'Predicted'
    value_name='Power Output' # Name for the new column holding the actual/predicted values
)

fig_compare_px = px.line(
    plot_df_long,
    x=timestamp_col_name,         # Use the timestamp column for x-axis
    y='Power Output',         # Use the combined value column for y-axis
    color='Source',           # Color lines based on 'Actual' or 'Predicted'
    title='Actual vs. Predicted Power Output (Test Set)',
    labels={timestamp_col_name: 'Timestamp', 'Power Output': 'Power Output (Units?)'}, # Clean labels
    template='plotly_dark',   # Or your preferred template
    # Optional: Customize line dash based on source
    line_dash='Source'
)

fig_compare_px.show()

ValueError: Length of values (5843) does not match length of index (35061)

## Export model


In [ ]:
# Access model parameters
model_name = config['models']['model_name']
model_output_path = os.path.join(model_name, config['models']['output_path'])
model_params = config['model_params']

In [ ]:
import jblib
import os

output_path = 
